## Instalação de bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.1 MB/s eta 0:00:00


In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

Definição da semente para aleatoriedade, para fins de reprodução do treinamento.

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Download e processamento da base de treinamento

In [ ]:
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2023-03-15 23:01:14--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 64.233.189.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.06s   

2023-03-15 23:01:14 (121 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('msmarco_triples.train.tiny.tsv', sep='\t', header=None)

In [ ]:
df.head()

,0,1,2
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [ ]:
df_pos = pd.DataFrame()
df_neg = pd.DataFrame()

for index, row in df.iterrows():
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0}, ignore_index=True)
    

Streaming output truncated to the last 5000 lines.
<ipython-input-10-42bb858fdfd1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
<ipython-input-10-42bb858fdfd1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
<ipython-input-10-42bb858fdfd1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
<ipython-input-10-42bb858fdfd1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df

In [ ]:
from sklearn.model_selection import train_test_split

X_train_pos = df_pos.drop("score", axis=1)
Y_train_pos = df_pos["score"]

X_train_pos, X_val_pos, Y_train_pos, Y_val_pos = train_test_split(X_train_pos, Y_train_pos, test_size=0.2, random_state=42)

X_train_neg = df_neg.drop("score", axis=1)
Y_train_neg = df_neg["score"]

X_train_neg, X_val_neg, Y_train_neg, Y_val_neg = train_test_split(X_train_neg, Y_train_neg, test_size=0.2, random_state=42)

In [ ]:
X_train = pd.concat([X_train_pos, X_train_neg], axis=0, ignore_index=True)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis=0, ignore_index=True)
X_val = pd.concat([X_val_pos, X_val_neg], axis=0, ignore_index=True)
Y_val = pd.concat([Y_val_pos, Y_val_neg], axis=0, ignore_index=True)

## Finetuning

Preparação dos dados, com tokenização e construção do dataset e dataloaders.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Testes iniciais com o modelo nreimers/MiniLM-L6-H384-uncased sugerido no enunciado do exercício, ainda que a acurácia do classificador tenha sido cerca de 0.938 na base de validação, levaram, mesmo com 20 épocas de treinamento, a um desempenho inferior ao obtido pelo algoritmo BM25 (nDCG@10 = 0.427, enquanto o BM25, conforme visto no exercício da aula anterior, levou a um valor de 0.48 para a mesma métroca).  Por esse motivo, optou-se por seguir os testes com o modelo microsoft/MiniLM-L12-H384-uncased.

Entretanto, este tópico será revisitado posteriormente, em uma etapa de otimização de hiperparâmetros.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
#miniLM (modelo BERT pequeno, 5x mais rápido): 
#https://huggingface.co/nreimers/MiniLM-L6-H384-uncased pois oferece um bom 
#compromisso entre qualidade e velocidade.
#Este aparentemente sofreu overfitting, com com 20 épocas de treinamento, embora 
#tenha atingido acurácia de validação igual a 0.938, levou a NDCG@10=0.427 (softmaxes)
#model_name = "nreimers/MiniLM-L6-H384-uncased"

#Este, com 5 épocas de finetuning, leva a uma acurácia de validação=0.937, e NDCG@10=0.621
model_name = 'microsoft/MiniLM-L12-H384-uncased' 



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(list(X_train["passage"])[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Mean length in tokens: 80.45
Stdev length in tokens: 31.20


In [ ]:
max_length = 512

In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

train_queries_tokenized = tokenizer(train_queries, max_length=max_length, truncation=True)
train_passages_tokenized = tokenizer(train_passages, max_length=max_length, truncation=True)
val_queries_tokenized = tokenizer(val_queries, max_length=max_length, truncation=True)
val_passages_tokenized = tokenizer(val_passages, max_length=max_length, truncation=True)

In [ ]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, queries, passages, targets):
        self.queries = queries
        self.passages = passages
        self.targets = targets
    
    def __len__(self):
        return len(self.queries['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.queries['input_ids'][idx] + self.passages['input_ids'][idx],
            'attention_mask': self.queries['attention_mask'][idx] + self.passages['attention_mask'][idx],
            'labels': int(self.targets[idx]),
        }



In [ ]:
dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == int

In [ ]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [ ]:
from transformers import BatchEncoding

# Tokens do tipo "pad" para textos com tamanho inferior ao máximo suportado.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))



Confirmando se os datasets de treinamento e validação gerados são sempre os mesmos.

In [ ]:
X_train.head()

,query,passage
0,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
1,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
2,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
4,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."


In [ ]:
X_val.head()

,query,passage
0,where to get corso riggs,Background. Always cheerful and eternally opti...
1,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
2,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
4,what is a pole pocket banner,Pole pockets on banners are something that can...


In [ ]:
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


### Training Loop

In [ ]:
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [ ]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train(model_name, epochs = 5, lr=5e-5):
  model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
  print('Parameters', model.num_parameters())

  #O ideal aqui é fazer um grid search incluindo pelo menos [2e-5, 3e-5, 5e-5].
  #optimizer = optim.AdamW(model.parameters(), lr=5e-5) #1a. tentativa: NDCG@10 = 0.6223 (logits)
  optimizer = optim.AdamW(model.parameters(), lr)
  num_training_steps = epochs * len(dataloader_train)

  num_warmup_steps = int(num_training_steps * 0.1)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

  evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

  # Training loop
  for epoch in tqdm(range(epochs), desc='Epochs'):
      model.train()
      train_losses = []
      for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
          optimizer.zero_grad()
          outputs = model(**batch.to(device))
          loss = outputs.loss
          loss.backward()
          optimizer.step()
          scheduler.step()
          train_losses.append(loss.cpu().item())

      print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
      evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
  
  return model

In [ ]:
model_name

'microsoft/MiniLM-L12-H384-uncased'

In [ ]:
MODELS_PATH = '/content/gdrive/MyDrive/Unicamp-aula-3'

### Primeiro experimento

In [ ]:
model = train(model_name, 5)
model_name = model_name.replace('/','_')
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.04


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.247; Valid accuracy: 0.937


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

### Otimização da taxa de aprendizagem

Ainda que, devido a limitações de tempo, não tenha sido feita uma grid search mais extensa sobre taxas de aprendizagem e outros parâmetros, aqui se busca verificar se pequenas variações na taxa de aprendizagem na etapa de finetuning acarretam mudanças significativas no desempenho do ranqueador.  Como ponto de partida, foram utilizados os valores para taxa de aprendizagem citados em https://wandb.ai/amogkam/transformers/reports/Hyperparameter-Optimization-for-HuggingFace-Transformers--VmlldzoyMTc2ODI

In [ ]:
#2e-5
model = train(model_name, 5, lr=2e-5)
model_name = model_name.replace('/','_') + '_lr_2e-5'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.693; Valid accuracy: 0.511


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.08


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.232; Valid accuracy: 0.932


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

In [ ]:
#3e-5
model_name = 'microsoft/MiniLM-L12-H384-uncased' 
model = train(model_name, 5, lr=3e-5)
model_name = model_name.replace('/','_') + '_lr_3e-5'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.07


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.241; Valid accuracy: 0.931


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

## Etapa de reranqueamento (estágio 2)

Inicialmente, será analisado o primeiro modelo treinado, que obteve a melhor acurácia até o momento para o conjunto de validação (0.937).

In [ ]:
main_path = '/content/gdrive/MyDrive/Unicamp-aula-2/'

In [ ]:
topics = 'dl20'
qrels_eval = 'dl20-passage'
qrels_eval_file = 'pyserini/tools/topics-and-qrels/qrels.dl20-passage.txt'
topics_file = 'pyserini/tools/topics-and-qrels/topics.dl20.txt'

In [ ]:
#root_dir = '/content/gdrive/MyDrive/Unicamp-aula-3/'
root_dir = 'gdrive/MyDrive/Unicamp-aula-3/'

In [ ]:
#Carrega os ids e textos das queries em memória
df_queries = pd.read_csv(f'{root_dir}topics.dl20.small.tsv', sep='\t', header=None)

In [ ]:
collection_path = '/content/gdrive/MyDrive/Unicamp-aula-2/collections/msmarco-passage/collection.tsv'
df_passages = pd.read_csv(collection_path, sep='\t', header=None)

In [ ]:
#Dúvida: não sei se é realmente necessário carregar o mesmo tokenizador usado no 
#finetuning, porque creio que ele não foi modificado.
tokenizer = AutoTokenizer.from_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

In [ ]:
import pickle

tokenized_queries = None
tokenized_passages = None

with open(f"{root_dir}tok_queries_test.pickle", "rb") as f:
  print("Carregando queries de teste...")
  tokenized_queries = pickle.load(f) 

with open(f"{root_dir}tok_passages_test.pickle", "rb") as f:
  print("Carregando passagens de teste...")
  tokenized_passages = pickle.load(f) 

query_ids = []
queries = []
passage_ids = []
passages = []

with open(f'{root_dir}runs/run.dl20.bm25tuned.txt') as f:
  for line in f:
      fields = line.strip().split('\t')
      query_id = fields[0]
      query_ids.append(query_id)
      passage_id = fields[1]
      passage_ids.append(passage_id)
      
      if not tokenized_queries:
        query_text = df_queries[df_queries.iloc[:,0] == int(query_id)].iloc[0,1]
        queries.append(query_text)

      if not tokenized_passages:
        passage_text = df_passages[df_passages.iloc[:,0] == int(passage_id)].iloc[0,1]
        passages.append(passage_text)

if not tokenized_queries:
  tokenized_queries = tokenizer(queries, max_length=max_length, truncation=True)

  with open(f"{root_dir}tok_queries_test.pickle", 'wb') as f:
    pickle.dump(tokenized_queries, f)

if not tokenized_passages:
  tokenized_passages = tokenizer(passages, max_length=max_length, truncation=True)

  with open(f"{root_dir}tok_passages_test.pickle", 'wb') as f:
    pickle.dump(tokenized_passages, f)
      

Carregando queries de teste...
Carregando passagens de teste...


In [ ]:
# Aqui, pouco importa o target, porque ele não será utilizado para calcular acurácia.  Por isso coloquei todos iguais 
# a 1 (relevantes)
dataset_test = Dataset(tokenized_queries, tokenized_passages, [1]*len(tokenized_queries['input_ids']))

In [ ]:
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [ ]:
def evaluate_test_dataset(model, dataloader, set_name, use_logits=False):
    scores = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            if use_logits:
              # Usa os logits brutos
              pos_score = outputs.logits[:,1]
            else:
              # Usa os logits normalizados pelo softmax (por default)
              pos_score = torch.softmax(outputs.logits,1)[:,1]
            scores = scores + pos_score.tolist()
    return scores

In [ ]:
def evaluate_ndcg_10(scores, model_name, eval_desc):
  zipped_results = []

  #Por alguma razão misteriosa, o zip do Python não funcionou, deixou a lista 
  #vazia ou impossível de ser iterada.
  for i, query_id in enumerate(query_ids):
    zipped_results.append((query_id, passage_ids[i], scores[i]))

  #Quebra a lista em sublistas por query
  prev_query_id = -1
  sublists = []
  current_list = []

  for query_id, passage_id, score in zipped_results:
    if query_id != prev_query_id:
      if len(current_list) > 0:
        sublists.append(current_list)
        current_list = []
    current_list.append((query_id, passage_id, score))
    prev_query_id = query_id

  if len(current_list) > 0:
    sublists.append(current_list)

  # Ordena cada sublista
  sorted_list = []

  for sublist in sublists:
    sorted_sublist = sorted(sublist, key=lambda x: x[2], reverse=True)
    sorted_list += sorted_sublist

  # Gera o arquivo de run no formato TREC
  trec_run_file = f"{root_dir}run.dl20.bert_reranked_{model_name}_{eval_desc}.trec"
  with open(trec_run_file, "w") as f:
    for i, (query_id, passage_id, score) in enumerate(sorted_list):
      f.write(f'{query_id}\tQ0\t{passage_id}\t{i+1}\t{score}\tbert_reranked_{model_name}\n')

  return trec_run_file
  

### Primeira avaliação

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)

In [ ]:
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
assert len(softmax_scores) == len(logit_scores) == 54000

Compara a avaliação do NDCG@10 utilizando softmax versus logits brutos.  É esperado que a avaliação softmax dê melhor resultado...

In [ ]:
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [ ]:
!chmod 755 {root_dir}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}


map                   	all	0.4340
ndcg_cut_10           	all	0.6210


... porém os logits levaram a uma ligeira vantagem para a métrica.  Cabem experimentos adicionais (ex.: teste de hipóteses) para checar esse dado.  Possivel explicação: inconsistências no dataset?

Na próxima seção, com a repetição dos experimentos (execuções de finetuning do classificador, sob diferentes inicializações da rede neural), esse fato será analisado novamente.

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.4338
ndcg_cut_10           	all	0.6223


### Reproduzindo o experimento mais 4 vezes para checar média/variância

In [ ]:

for i in range(1,4):
  model_name = 'microsoft/MiniLM-L12-H384-uncased' 
  model = train(model_name, 5)
  model_name = model_name.replace('/','_')
  model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}_{i}')
  tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

  softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
  logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)

  trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, f"softmax_{i}")
  trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, f"logits_{i}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.693; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch: 5 Training loss: 0.04


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.252; Valid accuracy: 0.930


Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
model_name

'microsoft_MiniLM-L12-H384-uncased'

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_softmax_0.trec

map                   	all	0.4055
ndcg_cut_10           	all	0.5889


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_softmax_1.trec

map                   	all	0.4183
ndcg_cut_10           	all	0.5849


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_softmax_2.trec

map                   	all	0.4147
ndcg_cut_10           	all	0.6053


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_softmax_3.trec

map                   	all	0.4159
ndcg_cut_10           	all	0.6214


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_logits_0.trec

map                   	all	0.4051
ndcg_cut_10           	all	0.5862


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_logits_1.trec

map                   	all	0.4183
ndcg_cut_10           	all	0.5851


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_logits_2.trec

map                   	all	0.4148
ndcg_cut_10           	all	0.6053


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {root_dir}run.dl20.bert_reranked_{model_name}_logits_3.trec

map                   	all	0.4162
ndcg_cut_10           	all	0.6217


#### Análise: softmax vs logits

Após mais 4 execuções de cada variação (uso de softmax vs logits), foram verificados os valores para a métrica NDCG@10 no total de 5 execuções, com cálculo de média e desvio-padrão.  Os valores são mostrados na tabela abaixo.  A conclusão é que não é possível afirmar que o uso dos logits leva a um melhor resultado em geral, dada a pouca diferença no valor médio para a média e ligeira variação para maior do desvio-padrão.

|| Softmax | Logits |
|---| --- | --- | 
|| 0.6210 | **0.6223** | 
|| **0.5889** | 0.5862 | 
|| 0.5849 | **0.5851** |
|| **0.6053** | **0.6053** |
|| 0.6214 | **0.6217** |
|**Média**|**0.6043**|**0,6041**|
|**Desvio-padrão**|**0.0172**|**0.0182**|



### Otimização de hiperparâmetros - taxas de aprendizagem

A seguir, serão analisados os outros dois modelos treinados com diferentes taxas de aprendizagem, que a princípio obtiveram acurácia no conjunto de validação ligeiramente inferior ao primeiro modelo treinado/analisado.

LR = 2e-5:

In [ ]:
model_name

'microsoft_MiniLM-L12-H384-uncased_lr_2e-5'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}x


map                   	all	0.4324
ndcg_cut_10           	all	0.6316


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.4322
ndcg_cut_10           	all	0.6315


LR = 3e-5:

In [ ]:
model_name

'microsoft_MiniLM-L12-H384-uncased_lr_3e-5'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}


map                   	all	0.4348
ndcg_cut_10           	all	0.6184


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.4351
ndcg_cut_10           	all	0.6196


**Conclusão**: a busca em um espaço de 3 taxas de aprendizagem, apesar de no fim ter levado ao modelo "vencedor" dentre aqueles aqui testados (LR = 2e-5) **não** levou a variações muito significativas para a métrica NDCG@10.

### Otimização de taxas de aprendizagem para modelo nreimers/MiniLM-L6-H384-uncased

Ainda que o modelo nreimers/MiniLM-L6-H384-uncased tenha revelado em pricípio um desempenho insatisfatório para a tarefa de ranqueamento, ainda assim ele será reanalisado nesta seção com a mesma variação de taxa de aprendizagem que foi analisada para o modelo microsoft/MiniLM-L12-H384.

In [ ]:
#2e-5
model_name = "nreimers/MiniLM-L6-H384-uncased"
model = train(model_name, 5, lr=2e-5)
model_name = model_name.replace('/','_') + '_lr_2e-5'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nreimers/MiniLM-L6-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 22713986


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.693; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.16


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.265; Valid accuracy: 0.901


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

In [ ]:
model_name

'nreimers_MiniLM-L6-H384-uncased_lr_2e-5'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
!chmod 755 {root_dir}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}

map                   	all	0.3601
ndcg_cut_10           	all	0.5505


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.3601
ndcg_cut_10           	all	0.5511


In [ ]:
#3e-5
model_name = "nreimers/MiniLM-L6-H384-uncased"
model = train(model_name, 5, lr=3e-5)
model_name = model_name.replace('/','_') + '_lr_3e-5'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nreimers/MiniLM-L6-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 22713986


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.693; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.11


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.265; Valid accuracy: 0.912


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

In [ ]:
model_name

'nreimers_MiniLM-L6-H384-uncased_lr_3e-5'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}

map                   	all	0.3553
ndcg_cut_10           	all	0.5191


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.3550
ndcg_cut_10           	all	0.5183


**Análise**: Apesar do ganho de cerca de 13 pontos percentuais quando o modelo assumiu uma taxa de aprendizagem de 2*e-5 ao invés de 5*e-5, ainda assim está inferior ao modelo microsoft/MiniLM-L12-H384-uncased, de maior capacidade.

### Investigação sobre overfitting

Finalmente, para o melhor modelo obtido até o momento, uma vez que durante o treinamento foi verificada uma tendência de aumento do erro de validação a partir da 4a. época (apesar do respectivo aumento na acurácia sobre a base de validação), será verificado se uma abordagem de *early stopping* após a terceira época pode levar a melhores resultados na tarefa de ranqueamento.

In [ ]:
#2e-5
model_name = 'microsoft/MiniLM-L12-H384-uncased' 
model = train(model_name, 3, lr=2e-5)
model_name = model_name.replace('/','_') + '_lr_2e-5_3_epochs'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Valid loss: 0.694; Valid accuracy: 0.500


...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 3 Training loss: 0.16


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.223; Valid accuracy: 0.925


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

In [ ]:
model_name

'microsoft_MiniLM-L12-H384-uncased_lr_2e-5_3_epochs'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [ ]:
!chmod 755 {root_dir}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}

map                   	all	0.4448
ndcg_cut_10           	all	0.6492


In [ ]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.4447
ndcg_cut_10           	all	0.6489


Conforme demonstrado pelos valores assumidos pela métrica nDCG@10, de fato a possibilidade de sobreajuste aos dados de validação é plausível.  Ainda que tenha havido uma diminuição na acurácia de validação em relação à versão do modelo treinada com 5 épocas, a métrica utilzada para o ranqueamento teve uma melhora de cerca de 1 ponto percentual.

Como entre a segunda e a terceira época houve um ligeiro aumento no erro de validação (entropia cruzada), vamos a seguir verificar se a técnica de early stoping ainda pode trazer mais vantagens antecipando o treinamento para apenas 2 épocas.

In [ ]:
#2e-5
model_name = 'microsoft/MiniLM-L12-H384-uncased' 
model = train(model_name, 2, lr=2e-5)
model_name = model_name.replace('/','_') + '_lr_2e-5_2_epochs'
model.save_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

...

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 2 Training loss: 0.20


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.225; Valid accuracy: 0.914


('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')

In [ ]:
model_name

'microsoft_MiniLM-L12-H384-uncased_lr_2e-5_2_epochs'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker_{model_name}').to(device)
softmax_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')
logit_scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test', use_logits=True)
trec_file_softmax = evaluate_ndcg_10(softmax_scores, model_name, "softmax")
trec_file_logits = evaluate_ndcg_10(logit_scores, model_name, "logits")

In [55]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_softmax}

map                   	all	0.4477
ndcg_cut_10           	all	0.6549


In [56]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec {trec_file_logits}

map                   	all	0.4479
ndcg_cut_10           	all	0.6536


De fato, houve um ganho ainda maior para a métrica final nDCG@10, e ao menos durante o treinamento, não foram verificados indícios visíveis de overfiting.  Além disso, a reducão do número de épocas de 5 para 2 tem a vantagem óbvia de levar a um menor tempo de treinamento.

Análises futuras, incluindo o regime de treinamento com 2 épocas, podem incluir:

*   repetição dos experimentos com taxas variáveis de aprendizagem
*   repetição dos experimentos com modelo nreimers/MiniLM-L6-H384-uncased, bem como outros modelos

